# Calculate PDF(x) and Parton Luminosity

Use the `parton` package:  https://github.com/DavidMStraub/parton

Install the `parton` package: `pip install parton`

Install PDF sets. For example: `python3 -m parton install 'CT18NNLO'`

A list of PDF sets for download are available at: https://www.lhapdf.org/pdfsets.html 

In [1]:
import parton
import math

# Calculate $x f(x,Q)$

In [ ]:
# Calculate x*f(x)

# get PDF set
pdf = parton.mkPDF('CT18NNLO', 0)

# particle code: 0,1,2,3,4,5 for d,u,s,c,b ; gluon has code 21
# anti-quarks have minus sign
# calculates x f(x,Q)
x = 0.01
Q = 100 # GeV ; # factorization scale, 
# compare to Figure 8.3 (right) in the Book
print('Q: ',Q,' GeV')
print('x: ',x)
print('x*u(x): ',pdf.xfxQ(2, 0.01, Q=Q))
print('x*bbar(x): ',pdf.xfxQ(-5, 0.01, Q=Q))
print('x*b(x): ',pdf.xfxQ(5, 0.01, Q=Q))
print('x*g(x): ',pdf.xfxQ(21, 0.01, Q=Q))

Q:  100  GeV
x:  0.01
x*u(x):  0.7883036490944652
x*bbar(x):  0.22981313000849626
x*b(x):  0.22981313000849626
x*g(x):  7.960073481872382


# Calculate Parton Luminosity

For the calculation we use the formulas in the book in section 8.1.4 'Parton Luminosity'

We provide two functions:
- our own function `partonlumi`, which follows the procedure described in section 8.1.4, and makes a simple integration over rapidity y. 
- the function `plumi` using the functions provided by the parton package 

As explained in the book, in both cases we divide by $s$ and convert to `nbarn` using the 
factor $0.389E6~\text{GeV}^2~\text{nb}$.    

In [ ]:
def partonlumi(pdf, s, shat, parton_a, parton_b):
    """
    Calculate the parton luminosity

    For the parameters and the procedure see section 8.1.4 in the book.
    Return parton lumi in nb
    """
    to_nb = 0.389E6 # Gev^2 nb
    tau = shat/s
    mu = math.sqrt(shat) # factorization scale, use shat
    ymin = 0.5 * math.log(tau) # see section 8.1.1
    ymax = -0.5 * math.log(tau)
    nbin = 1000
    dy = (ymax-ymin)/nbin
    if parton_a == parton_b:
        fac = 0.5
    else:
        fac = 1.
    dLdtau = 0.
    for i in range(nbin): # do simple integration
        y = ymin + dy/2 + i*dy
        xA = math.sqrt(tau)*math.exp(y)
        xB = math.sqrt(tau)*math.exp(-y)
        xfaA = pdf.xfxQ(parton_a, xA, Q=mu)
        xfbB = pdf.xfxQ(parton_b, xB, Q=mu)
        xfbA = pdf.xfxQ(parton_b, xA, Q=mu)
        xfaB = pdf.xfxQ(parton_a, xB, Q=mu)
        dL  = (xfaA*xfbB + xfbA*xfaB) * dy 
        dLdtau += dL
    # apply factor for double counting, convert to nb
    # since xfxQ() gives x f(x), dL contains factors xA*xB = tau, therefore divide by tau 
    return fac * dLdtau * to_nb / tau / s

In [13]:
def plumi(pdf, s, shat, parton_a, parton_b):
    """
    Calculate parton luminosity with the PLumi.L(...)

    Return parton lumi in nb
    """
    to_nb = 0.389E6 # Gev^2 nb
    tau = shat/s
    pl = parton.PLumi(pdf, Q2=shat) # use factorization scale Q2 = shat 
    if parton_a == parton_b:
        fac = 0.5
    else:
        fac = 1.
    pl2 = pl.L(parton_a, parton_b, tau) + pl.L(parton_b, parton_a, tau)
    return fac * pl2 * to_nb / s

# Examples

### Gluon-Gluon

In [16]:
# Check values in Figure 8.5 (gg) at 14 TeV
s = (1.4E4)**2 # sqrt(s)=14 TeV
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
lQ = [ 50, 100, 500, 1000]
parton_a = 21
parton_b = 21
pdf = parton.mkPDF('CT18NNLO', 0)
print('sqrt(shat)   partonlumi     plumi')
for Q in lQ:
    shat = Q*Q 
    pl1 = partonlumi(pdf, s, shat, parton_a, parton_b)
    pl2 = plumi(pdf, s, shat, parton_a, parton_b)
    print(f'{Q:>6.0f} GeV  {pl1:10.3e} nb  {pl2:10.3e} nb')


sqrt(s): 14000 GeV
sqrt(shat)   partonlumi     plumi
    50 GeV   1.595e+05 nb   1.594e+05 nb
   100 GeV   1.726e+04 nb   1.725e+04 nb
   500 GeV   3.324e+01 nb   3.321e+01 nb
  1000 GeV   1.156e+00 nb   1.155e+00 nb


In [17]:
# Check values in Figure 8.5 (gg) at 2 TeV
s = (2E3)**2 # sqrt(s)=2 TeV
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
lQ = [ 50, 100, 500, 1000]
parton_a = 21
parton_b = 21
pdf = parton.mkPDF('CT18NNLO', 0)
print('sqrt(shat)   partonlumi     plumi')
for Q in lQ:
    shat = Q*Q 
    pl1 = partonlumi(pdf, s, shat, parton_a, parton_b)
    pl2 = plumi(pdf, s, shat, parton_a, parton_b)
    print(f'{Q:>6.0f} GeV  {pl1:10.3e} nb  {pl2:10.3e} nb')

sqrt(s): 2000 GeV
sqrt(shat)   partonlumi     plumi
    50 GeV   8.134e+03 nb   8.126e+03 nb
   100 GeV   4.308e+02 nb   4.303e+02 nb
   500 GeV   2.509e-02 nb   2.506e-02 nb
  1000 GeV   9.258e-06 nb   9.249e-06 nb


### $u \bar{d}$ parton luminosity

In [18]:
# Check values in Figure 8.5 (udbar) at 14 TeV
s = (1.4E4)**2 # sqrt(s)=14 TeV
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
lQ = [ 50, 100, 500, 1000]
parton_a = 2
parton_b = -1
pdf = parton.mkPDF('CT18NNLO', 0)
print('sqrt(shat)   partonlumi     plumi')
for Q in lQ:
    shat = Q*Q 
    pl1 = partonlumi(pdf, s, shat, parton_a, parton_b)
    pl2 = plumi(pdf, s, shat, parton_a, parton_b)
    print(f'{Q:>6.0f} GeV  {pl1:10.3e} nb  {pl2:10.3e} nb')

sqrt(s): 14000 GeV
sqrt(shat)   partonlumi     plumi
    50 GeV   2.652e+03 nb   2.650e+03 nb
   100 GeV   4.002e+02 nb   3.998e+02 nb
   500 GeV   2.833e+00 nb   2.830e+00 nb
  1000 GeV   2.339e-01 nb   2.337e-01 nb


In [19]:
# Check values in Figure 8.5 (udbar) at 14 TeV
s = (2E3)**2 # sqrt(s)=2 TeV
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
lQ = [ 50, 100, 500, 1000]
parton_a = 2
parton_b = -1
pdf = parton.mkPDF('CT18NNLO', 0)
print('sqrt(shat)   partonlumi     plumi')
for Q in lQ:
    shat = Q*Q 
    pl1 = partonlumi(pdf, s, shat, parton_a, parton_b)
    pl2 = plumi(pdf, s, shat, parton_a, parton_b)
    print(f'{Q:>6.0f} GeV  {pl1:10.3e} nb  {pl2:10.3e} nb')

sqrt(s): 2000 GeV
sqrt(shat)   partonlumi     plumi
    50 GeV   4.179e+02 nb   4.174e+02 nb
   100 GeV   4.537e+01 nb   4.533e+01 nb
   500 GeV   2.516e-02 nb   2.513e-02 nb
  1000 GeV   1.595e-05 nb   1.594e-05 nb


### $u \bar{d}$ parton luminosity at $p \bar{p}$ beams

In [20]:
# Check values in Figure 8.5 (udbar) for Tevatron
# this is a  proton-antiproton beam of 1.9 TeV
# dbar in antiproton is like d in proton
s = (1.9E3)**2 # sqrt(s)=2 TeV
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
lQ = [ 50, 100, 500, 1000]
parton_a = 2
parton_b = 1
pdf = parton.mkPDF('CT18NNLO', 0)
print('sqrt(shat)   partonlumi     plumi')
for Q in lQ:
    shat = Q*Q 
    pl1 = partonlumi(pdf, s, shat, parton_a, parton_b)
    pl2 = plumi(pdf, s, shat, parton_a, parton_b)
    print(f'{Q:>6.0f} GeV  {pl1:10.3e} nb  {pl2:10.3e} nb')

sqrt(s): 1900 GeV
sqrt(shat)   partonlumi     plumi
    50 GeV   6.104e+02 nb   6.098e+02 nb
   100 GeV   8.116e+01 nb   8.108e+01 nb
   500 GeV   1.637e-01 nb   1.635e-01 nb
  1000 GeV   3.470e-04 nb   3.466e-04 nb


### Drell-Yan, Figure 8.19

In [21]:
# Section 8.3, 
# Check whether the factor 5/18 gives the same result as 
# a calculation where each qqbar contribution is multiplied with it charge**2
# Calculate qqbar at 8 TeV where each parton contribution is multiplied by Q**2
s = (8E3)**2 # sqrt(s)=8 TeV
shat = 50**2
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
pdf = parton.mkPDF('CT18NNLO', 0)
print('sqrt(shat)   partonlumi     plumi')
pldd = partonlumi(pdf, s, shat, 1, -1) * 1/9
pluu = partonlumi(pdf, s, shat, 2, -2) * 4/9
plss = partonlumi(pdf, s, shat, 3, -3) * 1/9
plcc = partonlumi(pdf, s, shat, 4, -4) * 4/9
plbb = partonlumi(pdf, s, shat, 5, -5) * 1/9
pl = pldd+pluu+plss+plcc+plbb
print(f'Lumi: {pl*1000:10.3e} pb')

sqrt(s): 8000 GeV
sqrt(shat)   partonlumi     plumi
Lumi:  1.013e+06 pb


In [22]:
# Calculate qqbar at 8 TeV 
# calculate the sum of q-qbar and apply the factor 5/18
# the result shows that this method give good agreement with the 
# previous exact calculation
# also compare parton lumi with reading from the plot: 3.4E6 pb
# CT18NNLO gives slightly larger values than MSTW2008nlo68cl
s = (8E3)**2 # sqrt(s)=8 TeV
shat = 50**2
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
#pdf = parton.mkPDF('CT18NNLO', 0)
pdf = parton.mkPDF('MSTW2008nlo68cl', 0)
pldd = partonlumi(pdf, s, shat, 1, -1)
pluu = partonlumi(pdf, s, shat, 2, -2)
plss = partonlumi(pdf, s, shat, 3, -3)
plcc = partonlumi(pdf, s, shat, 4, -4)
plbb = partonlumi(pdf, s, shat, 5, -5)
pl = pldd+pluu+plss+plcc+plbb
print(f'Lumi: {pl*1000:10.3e} pb')
print(f'Lumi*5/18: {pl*5/18*1000:10.3e} pb')


sqrt(s): 8000 GeV
Lumi:  3.413e+06 pb
Lumi*5/18:  9.482e+05 pb


In [23]:
# calculate parton luminosity for Z production at 8 TeV, fig. 8.19
# compare parton lumi with the reading from the plot of 5.9E5 pb
# CT18NNLO gives slightly larger values than MSTW2008nlo68cl
s = (8E3)**2 # sqrt(s)=8 TeV
shat = 91.2**2
print(f'sqrt(s): {math.sqrt(s):.0f} GeV')
#pdf = parton.mkPDF('CT18NNLO', 0)
pdf = parton.mkPDF('MSTW2008nlo68cl', 0)
pldd = partonlumi(pdf, s, shat, 1, -1)
pluu = partonlumi(pdf, s, shat, 2, -2)
plss = partonlumi(pdf, s, shat, 3, -3)
plcc = partonlumi(pdf, s, shat, 4, -4)
plbb = partonlumi(pdf, s, shat, 5, -5)
pl = pldd+pluu+plss+plcc+plbb
print(f'Lumi: {pl*1000:10.3e} pb')

sqrt(s): 8000 GeV
Lumi:  5.953e+05 pb
